In [1]:
#Import libraries
import numpy as np

In [49]:
#Read information 

#matrix = [[5,9,8,10,1],
#          [9,3,10,1,8],
#          [9,4,5,8,6],
#          [4,8,8,7,2]]

matrix = [[24,23],
          [18,22],
          [21,4],
          [12,17],
          [28,24],
          [19,25]]

matrix = np.array(matrix)
print(matrix)
size = matrix.shape
n = size[0]
m = size[1]
print('Cantidad de jobs : {}'.format(n))
print('Cantidad de máquinas : {}'.format(m))

[[24 23]
 [18 22]
 [21  4]
 [12 17]
 [28 24]
 [19 25]]
Cantidad de jobs : 6
Cantidad de máquinas : 2


In [50]:
#Hacer lo mismo pero con BBDD de github

In [51]:
#Entrega el tiempo que la primera tarea asignada termina en cada máquina

def get_times( indi , times ):
  aval = np.zeros(len(times))
  aval[0] = times[0]
  for i in range ( 1, len(times)):
    aval[i] = aval[i-1] + times[i]
  print('Tiempos de término de la tarea : {}\n{}'.format(indi,aval))
  return aval

In [52]:
#Entrega el tiempo que la nueva tarea asignada termina en cada máquina

def get_times2 ( indi , times , aval ):
  new_aval = np.zeros(len(times))
  new_aval[0] = aval[0] + times[0]
  for i in range ( 1, len(times)):
    new_aval[i] = max( new_aval[i-1] , aval[i] ) + times[i] 
  print('Tiempos de termino al agregar tarea : {}\n{}'.format( indi , new_aval))
  return new_aval

In [53]:
#Agrega la nueva tarea en la primera posición

def first ( indi_add , time , solution , matrix ):
  print('Agrega la tarea {} al principio'.format(indi_add))
  aval = get_times( indi_add , matrix[int(indi_add) -1] )
  for i in range ( 0 , len(solution)):
    aval = get_times2( solution[i] , matrix[solution[i] -1] , aval )
  return 0 , aval[len(aval) -1]

In [54]:
#Agrega la nueva tarea en una posición en el medio, específicamente en mid

def middle ( indi_add , time , solution , matrix , mid ):
  print('Agrega la tarea {} en posición {}'.format(indi_add,mid))
  aval = get_times( int(solution[0]) , matrix[int(solution[0]) -1] )
  sol = 1
  for i in range ( 1 , len(solution) +1):
    if ( i == mid ): aval = get_times2( indi_add , matrix[int(indi_add) -1] , aval )
    else: 
      aval = get_times2( solution[sol] , matrix[solution[sol] -1] , aval )
      sol += 1
  return mid , aval[len(aval) -1]

In [55]:
#Agrega la nueva tarea en la última posición

def last ( indi_add , time , solution , matrix ):
  print('Agrega la tarea {} al final'.format(indi_add))
  aval = get_times( int(solution[0]) , matrix[int(solution[0]) -1] )
  for i in range ( 1 , len(solution)):
    aval = get_times2( solution[i] , matrix[solution[i] -1] , aval )
  aval = get_times2( int(indi_add) , matrix[int(indi_add) -1] , aval )
  return len(solution) , aval[len(aval) -1]

In [56]:
#Create variables 
counter = 2      #how many jobs have been assiged 
indi = np.zeros(n)   #index of the jobs
for i in range (0,n):
  indi[i] = i +1

solution = []    #Save solution
Ftime = 0        #Final makespam

#Get time from each job in all machines
Time = np.zeros(n)
for i in range (0,n):
  Time[i] = sum(matrix[i])

#Get the decending orden of the jobs Time 
indiSorted = [x for _,x in sorted(zip(Time,indi),reverse = True)]
Time = np.sort(Time)[::-1]

print('Job sorted by time    {}\n Time in all machines {}'.format(indiSorted,Time))

Job sorted by time    [5.0, 1.0, 6.0, 2.0, 4.0, 3.0]
 Time in all machines [52. 47. 44. 40. 29. 25.]


In [57]:
def firstRun( indi_1 , indi_2 , matrix , counter):
  time_1 = matrix[int(indi_1) -1]
  time_2 = matrix[int(indi_2) -1]

  makespam = 0 

  for i in range ( 0, counter ):
    if ( i == 0):
      aval = get_times( indi_1 , matrix[int(indi_1) -1] ) #time when job is done in each machine
      new_aval = get_times2( int(indi_2) , matrix[int(indi_2) -1] , aval )
      solution.append(int(indi_1))
      makespam = new_aval[m-1]

    else:
      aval = get_times( indi_2 , matrix[int(indi_2) -1] ) #time when job is done in each machine
      new_aval = get_times2( int(indi_1) , matrix[int(indi_1) -1] , aval )
      if ( new_aval[m-1] < makespam ): 
        solution.insert(0,int(indi_2))
        makespam = new_aval[m-1] 
      else:
        solution.insert(1,int(indi_2))

  counter += 1
  return counter, solution

counter,solution = firstRun( indiSorted[0] , indiSorted[1] , matrix , counter)

Tiempos de término de la tarea : 5.0
[28. 52.]
Tiempos de termino al agregar tarea : 1
[52. 75.]
Tiempos de término de la tarea : 1.0
[24. 47.]
Tiempos de termino al agregar tarea : 5
[52. 76.]


In [58]:
while ( counter <= n ):
  print('----------------------- Entra a iteración -----------------------')  
  def otherRun( indi_add , matrix , counter , solution ):

    print('Entra a agregar la {} tarea\n'.format(indi_add))
    Fmakespam = 0      #Minimo makespam de la iteración 
    Fposition = 0      #Posición que se va a agregar la nueva tarea

    time = matrix[int(indi_add) -1]

    for i in range ( 0 , counter ):
      #add in first place
      if ( i == 0 ): 
        position , makespam = first( indi_add , time , solution , matrix )
        Fmakespam,Fposition = makespam,position

      #add in the middle
      elif ( i > 0 and i < counter -1 ): 
        position , makespam = middle( indi_add , time , solution , matrix , i )
        print(makespam,position)
      #add in last place
      elif ( i == counter -1 ): 
        position , makespam = last( indi_add , time , solution , matrix )
        print(makespam,position)
      if ( makespam < Fmakespam ): Fmakespam,Fposition = makespam,position

      print('\nMejor makespam hasta ahora es {} asignando tarea {} en posición {}\n'.format(Fmakespam , indi_add , Fposition))

    solution.insert( Fposition , indi_add )

    print('Solución de la Iteración: Asigna tarea {} en posición {} con makespam {}'.format( indi_add , Fposition , Fmakespam ))
    print('Nueva solución :\n{}'.format(solution))

    counter += 1

    return counter , solution , Fmakespam

  counter , solution , Ftime = otherRun( int(indiSorted[counter -1]) , matrix , counter , solution )
  print('---------------- Termina Iteración ----------------')
print('Solución Final es : \n{}\nCon makespam de {}'.format(solution,Ftime))

----------------------- Entra a iteración -----------------------
Entra a agregar la 6 tarea

Agrega la tarea 6 al principio
Tiempos de término de la tarea : 6
[19. 44.]
Tiempos de termino al agregar tarea : 5
[47. 71.]
Tiempos de termino al agregar tarea : 1
[71. 94.]

Mejor makespam hasta ahora es 94.0 asignando tarea 6 en posición 0

Agrega la tarea 6 en posición 1
Tiempos de término de la tarea : 5
[28. 52.]
Tiempos de termino al agregar tarea : 6
[47. 77.]
Tiempos de termino al agregar tarea : 1
[ 71. 100.]
100.0 1

Mejor makespam hasta ahora es 94.0 asignando tarea 6 en posición 0

Agrega la tarea 6 al final
Tiempos de término de la tarea : 5
[28. 52.]
Tiempos de termino al agregar tarea : 1
[52. 75.]
Tiempos de termino al agregar tarea : 6
[ 71. 100.]
100.0 2

Mejor makespam hasta ahora es 94.0 asignando tarea 6 en posición 0

Solución de la Iteración: Asigna tarea 6 en posición 0 con makespam 94.0
Nueva solución :
[6, 5, 1]
---------------- Termina Iteración ----------------
--